In [20]:
import os

FILE_PATH = os.path.abspath('__file__')
BASE_DIR = os.path.dirname(FILE_PATH)
TEMPLATE_DIR = os.path.join(BASE_DIR, "templates")

class Template:
    template_name = ''
    context = None
    
    def __init__(self, template_name="name", context=None, *args, **kwargs):
        self.template_name = template_name
        self.context = context
    
    def get_template(self):
        template_path = os.path.join(TEMPLATE_DIR, self.template_name)
        if not os.path.exists(template_path):
            raise Exception('This path does not exist')
        template_string = ''
        with open(template_path, 'r') as f:
            template_string = f.read()
        return template_string
    
    def render(self, context=None):
        render_ctx = context
        if self.context != None:
            render_ctx = self.context
        if not isinstance(render_ctx, dict):
            render_ctx = {}
        template_string = self.get_template()
        return template_string.format(**render_ctx)

In [21]:
tmpl = Template(template_name='hello.txt', context={'name': 'Cheetah'})
tmpl.get_template()
tmpl.render()

"Hello Cheetah. I'm going to lick you honey!"

In [22]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# environment variables
username = ''
password = ''


class Emailer():
    subject = ""
    context = {}
    to_emails = []
    has_html = False
    test_send = False
    from_email  = ''
    template_html = None
    template_name = None
    
    
    def __init__(self, subject="", template_name=None, context={}, template_html=None, to_emails=None, test_send=False):
        if template_name == None and template_html == None:
            raise Exception("You must set a template")
        assert isinstance(to_emails, list)
        self.to_emails = to_emails
        self.subject = subject
        if template_html != None:
            self.has_html = True
            self.template_html = template_html
        self.template_name = template_name
        self.context = context
        self.test_send = test_send
        
    def format_msg(self):
        msg = MIMEMultipart('alternative')
        msg['From'] = self.from_email
        msg['To'] = ", ".join(self.to_emails)
        msg['Subject'] = self.subject

        if self.template_name != None:
            tmpl_str = Template(template_name=self.template_name, context=self.context)
            txt_part = MIMEText(tmpl_str.render(), 'plain')
            print(txt_part)
            msg.attach(txt_part)
        if self.template_html != None:
            tmpl_str = Template(template_name=self.template_html, context=self.context)
            html_part = MIMEText(tmpl_str.render(), 'html')
            print(html_part)
            msg.attach(html_part)
        msg_str = msg.as_string()
        return msg_str
    
    def send(self):
        msg = self.format_msg()
        did_send = False
        if not self.test_send:
            with smtplib.SMTP(host='smtp.gmail.com', port=587) as server:
                server.ehlo()
                server.starttls()
                server.login(username, password)
                try:
                    server.sendmail(msg)
                    did_send = True
                except:
                    did_send = False
        return did_send
    

In [23]:
obj = Emailer(subject='Hello Darling', context={'name': 'Cheetah'}, template_name='hello.txt', template_html='hello.html', to_emails=['artururoda@gmail.com'], test_send=False)
obj.send()

Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

Hello Cheetah. I'm going to lick you honey!
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

<h1>Hello Cheetah. I'm going to lick you honey!</h1>



False